In [1]:
!rm -rf ./mri-reconstruction
!git clone -q https://github.com/MohammadRaziei/mri-reconstruction > /dev/null
!cp -r mri-reconstruction/python/* . && rm -r mri-reconstruction/python > /dev/null

Cloning into 'mri-reconstruction'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 41 (delta 8), reused 31 (delta 3), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [28]:
!pip install -r requirements.txt > /dev/null
!pip install git+https://github.com/ismrmrd/ismrmrd-python-tools.git > /dev/null


  Running command git clone -q https://github.com/ismrmrd/ismrmrd-python-tools.git /tmp/pip-req-build-dn3nggd6


In [27]:
import os, glob, sys, io
from pprint import pprint
import numpy as np
import pandas as pd
import tensorflow as tf

import ismrmrd
import ismrmrdtools
import mridatapreprocessing as mdp

In [20]:
def toStr(x):
  if isinstance(x, bytes): return x.decode("utf-8")
  else: return str(x) 

In [7]:
data_dir = 'data'
dir_mridata_org = os.path.join(data_dir, 'raw/ismrmrd')
uuid = "cc52722b-8649-45b0-a1ea-8727c1687ad5"
filename = os.path.join(dir_mridata_org, uuid+".h5")
uuid_txt = io.StringIO(uuid)


In [6]:
mdp.download_mridata_org_dataset(uuid_txt, dir_mridata_org)

  0%|          | 70/2018525 [00:00<53:35, 627.63KB/s]

100%|██████████| 2018525/2018525 [01:03<00:00, 31848.00KB/s]


In [25]:
dataset = ismrmrd.Dataset(filename, create_if_needed=False)
header = ismrmrd.xsd.CreateFromDocument(dataset.read_xml_header())
num_kx = header.encoding[0].encodedSpace.matrixSize.x
num_ky = header.encoding[0].encodingLimits.kspace_encoding_step_1.maximum
num_slices = header.encoding[0].encodingLimits.slice.maximum + 1
num_channels = header.acquisitionSystemInformation.receiverChannels
num_acq = dataset.number_of_acquisitions()



In [ ]:
print(toStr(dataset.read_xml_header()))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Dense

In [ ]:
# import ismrmrdtools

In [ ]:
import ismrmrd, ismrmrd.xsd
import logging as logger
from pprint import pprint
logger.basicConfig(level=logging.INFO)

In [ ]:
def ismrmrd_to_np(filename):
    """Read ISMRMRD data file to numpy array"""
    logger.debug('Loading file {}...'.format(filename))
    dataset = ismrmrd.Dataset(filename, create_if_needed=False)
    header = ismrmrd.xsd.CreateFromDocument(dataset.read_xml_header())
    num_kx = header.encoding[0].encodedSpace.matrixSize.x
    num_ky = header.encoding[0].encodingLimits.kspace_encoding_step_1.maximum
    num_slices = header.encoding[0].encodingLimits.slice.maximum + 1
    num_channels = header.acquisitionSystemInformation.receiverChannels

In [ ]:
dataset = ismrmrd.Dataset(filename, create_if_needed=False)
dataset

In [ ]:
header = ismrmrd.xsd.CreateFromDocument(dataset.read_xml_header())
header

ismrmrdHeader(version=None, subjectInformation=None, studyInformation=studyInformationType(studyDate=None, studyTime=XmlTime(7, 56, 53, 0), studyID=None, accessionNumber=None, referringPhysicianName=None, studyDescription=None, studyInstanceUID=None), measurementInformation=measurementInformationType(measurementID='45167_59757578_59757587_1821', seriesDate=None, seriesTime=None, patientPosition=<patientPositionType.FFS: 'FFS'>, initialSeriesNumber=None, protocolName='SAG', seriesDescription=None, measurementDependency=[measurementDependencyType(dependencyType='SenMap', measurementID='45167_59757578_59757587_1815'), measurementDependencyType(dependencyType='Noise', measurementID='45167_59757578_59757587_1815')], seriesInstanceUIDRoot=None, frameOfReferenceUID='1.3.12.2.1107.5.2.19.45167.1.20160817074730264.0.0.0', referencedImageSequence=None), acquisitionSystemInformation=acquisitionSystemInformationType(systemVendor='SIEMENS', systemModel='Skyra', systemFieldStrength_T=2.89362, relati

In [ ]:
header.encoding

[encodingType(encodedSpace=encodingSpaceType(matrixSize=matrixSizeType(x=768, y=770, z=1), fieldOfView_mm=fieldOfViewMm(x=280.0, y=280.700012, z=4.5)), reconSpace=encodingSpaceType(matrixSize=matrixSizeType(x=384, y=384, z=1), fieldOfView_mm=fieldOfViewMm(x=140.0, y=140.0, z=3.0)), encodingLimits=encodingLimitsType(kspace_encoding_step_0=None, kspace_encoding_step_1=limitType(minimum=0, maximum=615, center=308), kspace_encoding_step_2=limitType(minimum=0, maximum=0, center=0), average=limitType(minimum=0, maximum=0, center=0), slice=limitType(minimum=0, maximum=35, center=0), contrast=limitType(minimum=0, maximum=0, center=0), phase=limitType(minimum=0, maximum=0, center=0), repetition=limitType(minimum=0, maximum=0, center=0), set=limitType(minimum=0, maximum=0, center=0), segment=limitType(minimum=0, maximum=0, center=0)), trajectory=<trajectoryType.CARTESIAN: 'cartesian'>, trajectoryDescription=None, parallelImaging=parallelImagingType(accelerationFactor=accelerationFactorType(kspac

In [24]:
import io
fake_file = io.StringIO("your text goes here")

In [28]:
type(fake_file)

_io.StringIO

In [31]:
isinstance(fake_file, io.StringIO)

True

In [ ]:
def download_mridata_org_dataset(filename_txt, dir_output):
    """Download datasets from mridata.org if needed"""
    if os.path.isdir(dir_output):
        logger.warning(
            'Downloading data mridata.org to existing directory {}...'.format(
                dir_output))
    else:
        os.makedirs(dir_output)
        logger.info(
            'Downloading data from mridata.org to {}...'.format(dir_output))

    uuids = open(filename_txt).read().splitlines()
    for uuid in uuids:
        if not os.path.exists('{}/{}.h5'.format(dir_output, uuid)):
            mridata.download(uuid, folder=dir_output)


In [48]:
logger = mdp.logger
import utils.logging
# logger.setLevel(utils.logging.logging.DEBUG)
from tqdm.auto import tqdm
def ismrmrd_to_np(filename):
    """Read ISMRMRD data file to numpy array"""
    logger.debug('Loading file {}...'.format(filename))
    dataset = ismrmrd.Dataset(filename, create_if_needed=False)
    header = ismrmrd.xsd.CreateFromDocument(dataset.read_xml_header())
    num_kx = header.encoding[0].encodedSpace.matrixSize.x
    num_ky = header.encoding[0].encodingLimits.kspace_encoding_step_1.maximum
    num_slices = header.encoding[0].encodingLimits.slice.maximum + 1
    num_channels = header.acquisitionSystemInformation.receiverChannels

    try:
        rec_std = dataset.read_array('rec_std', 0)
        rec_weight = 1.0 / (rec_std**2)
        rec_weight = np.sqrt(rec_weight / np.sum(rec_weight))
        logger.debug('  Using rec std...')
    except Exception:
        rec_weight = np.ones(num_channels)
    opt_mat = np.diag(rec_weight)
    kspace = np.zeros([num_channels, num_slices, num_ky, num_kx],
                      dtype=np.complex64)
    num_acq = dataset.number_of_acquisitions()

    def wrap(x):
        return x

    if logger.getEffectiveLevel() is utils.logging.logging.DEBUG:
        wrap = tqdm
    for i in wrap(range(num_acq//2-1)):
        acq = dataset.read_acquisition(i)
        i_ky = acq.idx.kspace_encode_step_1  # pylint: disable=E1101
        # i_kz = acq.idx.kspace_encode_step_2 # pylint: disable=E1101
        i_slice = acq.idx.slice  # pylint: disable=E1101
        data = np.matmul(opt_mat.T, acq.data)
        kspace[:, i_slice, i_ky, :] = data * ((-1)**i_slice)

    dataset.close()
    # kspace = fftc.fftc(kspace, axis=1)

    return kspace

In [47]:
ismrmrd_to_np(filename)

DEBUG:dl-cs:Loading file data/raw/ismrmrd/cc52722b-8649-45b0-a1ea-8727c1687ad5.h5...


IndexError: ignored

In [58]:
import ismrmrdtools.imageviewer
ismrmrdtools.imageviewer.read_ismrmrd_image_series(filename, '')

KeyError: ignored

In [61]:
https://github.com/ismrmrd/ismrmrd-python-tools/blob/master/recon_ismrmrd_dataset.py

data/raw/ismrmrd/cc52722b-8649-45b0-a1ea-8727c1687ad5.h5


In [ ]:
from mpi4py import MPI

ImportError: DLL load failed: The specified procedure could not be found.